# 1. Introdução

In [335]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, shapiro
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import warnings

In [336]:
df = pd.read_csv('https://raw.githubusercontent.com/VitoriaPontes/MesariosCeara/refs/heads/main/mesarios.csv')

In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34637 entries, 0 to 34636
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Estado civil                            34637 non-null  object
 1   Faixa etária                            34637 non-null  object
 2   Gênero                                  34637 non-null  object
 3   Grau de escolaridade                    34637 non-null  object
 4   Município                               34637 non-null  object
 5   País                                    34637 non-null  object
 6   Região                                  34637 non-null  object
 7   Tipo de convocação                      34637 non-null  object
 8   Turno                                   34637 non-null  int64 
 9   UF                                      34637 non-null  object
 10  Zona                                    34637 non-null  int64 
 11  Qu

In [338]:
display(df)

,Estado civil,Faixa etária,Gênero,Grau de escolaridade,Município,País,Região,Tipo de convocação,Turno,UF,Zona,Quantidade de mesários voluntários,Quantidade de mesários,Quantidade de mesários não voluntários,Comparecimento,Ausência,Data de carga
0,CASADO,18 anos,FEMININO,ENSINO MÉDIO COMPLETO,FORQUILHA,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,121,1,1,0,1,0,2025-02-24 05:32:02
1,CASADO,19 anos,FEMININO,ENSINO MÉDIO COMPLETO,ACARAÚ,Brasil,NORDESTE,Não voluntárias e não voluntários,1,CE,30,0,1,1,1,0,2025-02-24 05:32:02
2,CASADO,19 anos,FEMININO,ENSINO MÉDIO COMPLETO,UMIRIM,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,23,1,1,0,1,0,2025-02-24 05:32:02
3,CASADO,19 anos,FEMININO,ENSINO MÉDIO INCOMPLETO,FORTALEZA,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,93,1,1,0,0,1,2025-02-24 05:32:02
4,CASADO,19 anos,FEMININO,ENSINO MÉDIO INCOMPLETO,FORTIM,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,8,1,1,0,1,0,2025-02-24 05:32:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34632,VIÚVO,70 a 74 anos,FEMININO,ENSINO MÉDIO INCOMPLETO,FORTALEZA,Brasil,NORDESTE,Voluntárias e voluntários,2,CE,3,1,1,0,1,0,2025-02-24 05:32:02
34633,VIÚVO,70 a 74 anos,FEMININO,SUPERIOR COMPLETO,FORTALEZA,Brasil,NORDESTE,Não voluntárias e não voluntários,1,CE,1,0,2,2,2,0,2025-02-24 05:32:02
34634,VIÚVO,70 a 74 anos,FEMININO,SUPERIOR COMPLETO,FORTALEZA,Brasil,NORDESTE,Não voluntárias e não voluntários,2,CE,1,0,2,2,2,0,2025-02-24 05:32:02
34635,VIÚVO,70 a 74 anos,FEMININO,SUPERIOR COMPLETO,JUAZEIRO DO NORTE,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,28,1,1,0,1,0,2025-02-24 05:32:02


# 2. Entendimento dos dados

In [339]:
df['Gênero'].value_counts()

,count
Gênero,
FEMININO,21347
MASCULINO,13290


In [340]:
df['Ausência'].value_counts()

,count
Ausência,
0,32576
1,1720
2,248
3,60
4,20
5,12
7,1


In [341]:
df['Tipo de convocação'].value_counts()

,count
Tipo de convocação,
Voluntárias e voluntários,21786
Não voluntárias e não voluntários,12851


In [342]:
df['Faixa etária'].value_counts()

,count
Faixa etária,
35 a 39 anos,5357
40 a 44 anos,4983
30 a 34 anos,4865
45 a 49 anos,4123
25 a 29 anos,4051
50 a 54 anos,3011
21 a 24 anos,2364
55 a 59 anos,2118
60 a 64 anos,1095


# 3. Tratamento das variáveis

In [343]:
df_jovem = df[df['Faixa etária'].isin(['18 anos', '19 anos', '20 anos', '21 a 24 anos'])]

In [344]:
df_jovem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4617 entries, 0 to 34173
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Estado civil                            4617 non-null   object
 1   Faixa etária                            4617 non-null   object
 2   Gênero                                  4617 non-null   object
 3   Grau de escolaridade                    4617 non-null   object
 4   Município                               4617 non-null   object
 5   País                                    4617 non-null   object
 6   Região                                  4617 non-null   object
 7   Tipo de convocação                      4617 non-null   object
 8   Turno                                   4617 non-null   int64 
 9   UF                                      4617 non-null   object
 10  Zona                                    4617 non-null   int64 
 11  Quantida

In [345]:
df_jovem['Ausência'].value_counts()

,count
Ausência,
0,4179
1,370
2,44
3,15
4,6
5,3


In [346]:
df_jovem['Gênero'].value_counts()

,count
Gênero,
FEMININO,2734
MASCULINO,1883


In [347]:
df_jovem['Estado civil'].value_counts()

,count
Estado civil,
SOLTEIRO,4409
CASADO,188
DIVORCIADO,12
SEPARADO JUDICIALMENTE,4
VIÚVO,4


In [348]:
df_jovem['Tipo de convocação'].value_counts()

,count
Tipo de convocação,
Voluntárias e voluntários,2472
Não voluntárias e não voluntários,2145


In [349]:
display(df_jovem)

,Estado civil,Faixa etária,Gênero,Grau de escolaridade,Município,País,Região,Tipo de convocação,Turno,UF,Zona,Quantidade de mesários voluntários,Quantidade de mesários,Quantidade de mesários não voluntários,Comparecimento,Ausência,Data de carga
0,CASADO,18 anos,FEMININO,ENSINO MÉDIO COMPLETO,FORQUILHA,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,121,1,1,0,1,0,2025-02-24 05:32:02
1,CASADO,19 anos,FEMININO,ENSINO MÉDIO COMPLETO,ACARAÚ,Brasil,NORDESTE,Não voluntárias e não voluntários,1,CE,30,0,1,1,1,0,2025-02-24 05:32:02
2,CASADO,19 anos,FEMININO,ENSINO MÉDIO COMPLETO,UMIRIM,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,23,1,1,0,1,0,2025-02-24 05:32:02
3,CASADO,19 anos,FEMININO,ENSINO MÉDIO INCOMPLETO,FORTALEZA,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,93,1,1,0,0,1,2025-02-24 05:32:02
4,CASADO,19 anos,FEMININO,ENSINO MÉDIO INCOMPLETO,FORTIM,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,8,1,1,0,1,0,2025-02-24 05:32:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18591,SOLTEIRO,21 a 24 anos,MASCULINO,SUPERIOR INCOMPLETO,VÁRZEA ALEGRE,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,62,1,1,0,1,0,2025-02-24 05:32:02
34170,VIÚVO,18 anos,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,TIANGUÁ,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,81,1,1,0,1,0,2025-02-24 05:32:02
34171,VIÚVO,19 anos,FEMININO,ENSINO MÉDIO INCOMPLETO,BREJO SANTO,Brasil,NORDESTE,Voluntárias e voluntários,1,CE,70,1,1,0,1,0,2025-02-24 05:32:02
34172,VIÚVO,20 anos,FEMININO,ENSINO MÉDIO COMPLETO,ITAPIPOCA,Brasil,NORDESTE,Não voluntárias e não voluntários,1,CE,17,0,1,1,1,0,2025-02-24 05:32:02


In [350]:
df_jovem = df_jovem.drop(columns=['País', 'Região', 'UF', 'Data de carga'])

In [351]:
df_jovem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4617 entries, 0 to 34173
Data columns (total 13 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Estado civil                            4617 non-null   object
 1   Faixa etária                            4617 non-null   object
 2   Gênero                                  4617 non-null   object
 3   Grau de escolaridade                    4617 non-null   object
 4   Município                               4617 non-null   object
 5   Tipo de convocação                      4617 non-null   object
 6   Turno                                   4617 non-null   int64 
 7   Zona                                    4617 non-null   int64 
 8   Quantidade de mesários voluntários      4617 non-null   int64 
 9   Quantidade de mesários                  4617 non-null   int64 
 10  Quantidade de mesários não voluntários  4617 non-null   int64 
 11  Comparec

# One-Hot Encoding

In [352]:
# função one-hot encoding
def one_hot_encoding(df, coluna):
  dados_unicos = df[coluna].unique()

  for dado in dados_unicos:
    df[f'{coluna}_{dado}'] = df[coluna].apply(lambda x: int(dado in x))

  df.drop(columns=[coluna], inplace=True)

In [353]:
# aplicação do one-hot encoding
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
one_hot_encoding(df_jovem, 'Estado civil')
one_hot_encoding(df_jovem, 'Faixa etária')
one_hot_encoding(df_jovem, 'Gênero')
one_hot_encoding(df_jovem, 'Grau de escolaridade')

In [354]:
df_jovem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4617 entries, 0 to 34173
Data columns (total 27 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Município                                           4617 non-null   object
 1   Tipo de convocação                                  4617 non-null   object
 2   Turno                                               4617 non-null   int64 
 3   Zona                                                4617 non-null   int64 
 4   Quantidade de mesários voluntários                  4617 non-null   int64 
 5   Quantidade de mesários                              4617 non-null   int64 
 6   Quantidade de mesários não voluntários              4617 non-null   int64 
 7   Comparecimento                                      4617 non-null   int64 
 8   Ausência                                            4617 non-null   int64 
 9   Estado civil

# 4. Teste de Hipóteses

In [355]:
def classificar_variaveis(df):
  quantitativas = []
  qualitativas = []

  for coluna in df.columns:
      tipo_dado = df_jovem[coluna].dtype
      num_unicos = df_jovem[coluna].nunique()

      if pd.api.types.is_numeric_dtype(df[coluna]):
            quantitativas.append(coluna)
      else:
            qualitativas.append(coluna)

  return quantitativas, qualitativas

In [356]:
quantitativas, qualitativas = classificar_variaveis(df_jovem)

In [357]:
print("Quantitativas:", quantitativas)
print("Qualitativas:", qualitativas)

Quantitativas: ['Turno', 'Zona', 'Quantidade de mesários voluntários', 'Quantidade de mesários', 'Quantidade de mesários não voluntários', 'Comparecimento', 'Ausência', 'Estado civil_CASADO', 'Estado civil_DIVORCIADO', 'Estado civil_SEPARADO JUDICIALMENTE', 'Estado civil_SOLTEIRO', 'Estado civil_VIÚVO', 'Faixa etária_18 anos', 'Faixa etária_19 anos', 'Faixa etária_20 anos', 'Faixa etária_21 a 24 anos', 'Gênero_FEMININO', 'Gênero_MASCULINO', 'Grau de escolaridade_ENSINO MÉDIO COMPLETO', 'Grau de escolaridade_ENSINO MÉDIO INCOMPLETO', 'Grau de escolaridade_SUPERIOR INCOMPLETO', 'Grau de escolaridade_ENSINO FUNDAMENTAL COMPLETO', 'Grau de escolaridade_ENSINO FUNDAMENTAL INCOMPLETO', 'Grau de escolaridade_SUPERIOR COMPLETO', 'Grau de escolaridade_LÊ E ESCREVE']
Qualitativas: ['Município', 'Tipo de convocação']


In [358]:
remover = []

In [359]:
def teste_associacao(df_jovem, qualitativas, quantitativas):
    coluna_grupo = 'Tipo de convocação'

    resultados = {
        'qualitativas': [],
        'quantitativas': []
    }

    # Teste qui-quadrado para variáveis qualitativas
    for var in qualitativas:
        if var == coluna_grupo:
            continue
        cont_tabela = pd.crosstab(df_jovem[coluna_grupo], df[var])
        if cont_tabela.shape[0] > 1 and cont_tabela.shape[1] > 1:
            try:
                stat, p, _, _ = chi2_contingency(cont_tabela)
                resultados['qualitativas'].append((var, p))
            except ValueError:
                continue  # ignora casos com dados malformados

    # Verifica se existem exatamente 2 grupos para o teste t ou Mann-Whitney
    grupos = df_jovem[coluna_grupo].dropna().unique()
    if len(grupos) == 2:
        grupo1 = df_jovem[df_jovem[coluna_grupo] == grupos[0]]
        grupo2 = df_jovem[df_jovem[coluna_grupo] == grupos[1]]

        for var in quantitativas:
            x1 = grupo1[var].dropna()
            x2 = grupo2[var].dropna()

            # Prossegue apenas se ambos os grupos têm dados
            if len(x1) > 10 and len(x2) > 10:
                try:
                    # Teste de normalidade (limitando amostras para velocidade)
                    _, p1 = shapiro(x1.sample(n=min(500, len(x1))))
                    _, p2 = shapiro(x2.sample(n=min(500, len(x2))))

                    if p1 > 0.05 and p2 > 0.05:
                        stat, p = ttest_ind(x1, x2, equal_var=False)
                        metodo = "t de Student"
                    else:
                        stat, p = mannwhitneyu(x1, x2, alternative='two-sided')
                        metodo = "Mann-Whitney"

                    resultados['quantitativas'].append((var, p, metodo))
                except:
                    continue  # ignora erros ocasionais por dados ruins

    return resultados

In [360]:
resultados = teste_associacao(df_jovem, qualitativas, quantitativas)

print("Testes com variáveis qualitativas:")
for var, p in resultados['qualitativas']:
  if p > 0.05:
    remover.append(var)
  print(f"{var}: p = {p:.4f}")

print("\nTestes com variáveis quantitativas:")
for var, p, metodo in resultados['quantitativas']:
  if p > 0.05:
    remover.append(var)
  print(f"{var}: p = {p:.4f} ({metodo})")

Testes com variáveis qualitativas:
Município: p = 0.0000

Testes com variáveis quantitativas:
Turno: p = 0.0132 (Mann-Whitney)
Zona: p = 0.0128 (Mann-Whitney)
Quantidade de mesários voluntários: p = 0.0000 (Mann-Whitney)
Quantidade de mesários: p = 0.0134 (Mann-Whitney)
Quantidade de mesários não voluntários: p = 0.0000 (Mann-Whitney)
Comparecimento: p = 0.1047 (Mann-Whitney)
Ausência: p = 0.0000 (Mann-Whitney)
Estado civil_CASADO: p = 0.0062 (Mann-Whitney)
Estado civil_DIVORCIADO: p = 0.7390 (Mann-Whitney)
Estado civil_SEPARADO JUDICIALMENTE: p = 0.3895 (Mann-Whitney)
Estado civil_SOLTEIRO: p = 0.0052 (Mann-Whitney)
Estado civil_VIÚVO: p = 0.8876 (t de Student)
Faixa etária_18 anos: p = 0.8202 (Mann-Whitney)
Faixa etária_19 anos: p = 0.8452 (Mann-Whitney)
Faixa etária_20 anos: p = 0.1523 (Mann-Whitney)
Faixa etária_21 a 24 anos: p = 0.1446 (Mann-Whitney)
Gênero_FEMININO: p = 0.5160 (Mann-Whitney)
Gênero_MASCULINO: p = 0.5160 (Mann-Whitney)
Grau de escolaridade_ENSINO MÉDIO COMPLETO: p

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [361]:
for coluna in remover:
  if coluna in df_jovem:
    df_jovem.drop(columns=[coluna], inplace=True)

In [362]:
df_jovem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4617 entries, 0 to 34173
Data columns (total 12 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Município                                     4617 non-null   object
 1   Tipo de convocação                            4617 non-null   object
 2   Turno                                         4617 non-null   int64 
 3   Zona                                          4617 non-null   int64 
 4   Quantidade de mesários voluntários            4617 non-null   int64 
 5   Quantidade de mesários                        4617 non-null   int64 
 6   Quantidade de mesários não voluntários        4617 non-null   int64 
 7   Ausência                                      4617 non-null   int64 
 8   Estado civil_CASADO                           4617 non-null   int64 
 9   Estado civil_SOLTEIRO                         4617 non-null   int64 
 10  Grau

In [363]:
df_jovem.drop(columns=['Município'], inplace=True)

# 5. Partição dos dados

In [364]:
X = df_jovem.drop('Tipo de convocação', axis=1)
y = df_jovem['Tipo de convocação']

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y
)

print(f'Tamanho treino: {X_train.shape}')
print(f'Tamanho validação: {X_val.shape}')
print("\nDistribuição em y_train:")
print(y_train.value_counts(normalize=True))
print("\nDistribuição em y_val:")
print(y_val.value_counts(normalize=True))

Tamanho treino: (3693, 10)
Tamanho validação: (924, 10)

Distribuição em y_train:
Tipo de convocação
Voluntárias e voluntários            0.535337
Não voluntárias e não voluntários    0.464663
Name: proportion, dtype: float64

Distribuição em y_val:
Tipo de convocação
Voluntárias e voluntários            0.535714
Não voluntárias e não voluntários    0.464286
Name: proportion, dtype: float64


In [365]:
y_train.value_counts()

,count
Tipo de convocação,
Voluntárias e voluntários,1977
Não voluntárias e não voluntários,1716


In [366]:
y_val.value_counts()

,count
Tipo de convocação,
Voluntárias e voluntários,495
Não voluntárias e não voluntários,429


In [367]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

# 6. Aprendizado de máquina

In [368]:
models = []

In [369]:
X_train.columns = X_train.columns.str.replace(r"[<>\[\]\(\)]", "", regex=True)
X_val.columns = X_val.columns.str.replace(r"[<>\[\]\(\)]", "", regex=True)

modelos_manuais = {
    "Regressão Logística": LogisticRegression(max_iter=1000, random_state=42),
    "Árvore de Decisão": DecisionTreeClassifier(max_depth=5, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=7, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.01, max_depth=4, random_state=42,
                             use_label_encoder=False, eval_metric='mlogloss'),
    "Rede Neural (MLP)": MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=500, random_state=42)
}

resultados_manuais = {}

for nome, modelo in modelos_manuais.items():
    print(f"Treinando modelo: {nome}")
    modelo.fit(X_train, y_train)
    pred_train = modelo.predict(X_train)
    pred_val = modelo.predict(X_val)

    models.append(modelo)

    resultados_manuais[nome] = {
        "Accuracy treino": accuracy_score(y_train, pred_train),
        "Accuracy validação": accuracy_score(y_val, pred_val),
        "Relatório de classificação": classification_report(y_val, pred_val, output_dict=True)
    }

df_resultados = pd.DataFrame({
    nome: {
        "Accuracy treino": valores["Accuracy treino"],
        "Accuracy validação": valores["Accuracy validação"]
    }
    for nome, valores in resultados_manuais.items()
}).T.sort_values("Accuracy validação", ascending=False)

df_resultados

Treinando modelo: Regressão Logística
Treinando modelo: Árvore de Decisão
Treinando modelo: Random Forest
Treinando modelo: XGBoost
Treinando modelo: Rede Neural (MLP)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:51:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


,Accuracy treino,Accuracy validação
Regressão Logística,1.0,1.0
Árvore de Decisão,1.0,1.0
Random Forest,1.0,1.0
XGBoost,1.0,1.0
Rede Neural (MLP),1.0,1.0


In [370]:
models_optimazed = []

In [333]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

X_train.columns = X_train.columns.str.replace(r"[<>\[\]\(\)]", "", regex=True)
X_val.columns = X_val.columns.str.replace(r"[<>\[\]\(\)]", "", regex=True)

melhores_modelos = {}
resultados_otimizados = {}
models_optimazed = []

param_logreg = {
    'C': uniform(0.01, 10),
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': [100, 200, 500, 1000]
}

param_tree = {
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

param_rf = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 15),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

param_xgb = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.5, 0.5)
}

param_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (50, 50, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': uniform(0.0001, 0.01),
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [300, 500, 1000]
}

rs_logreg = RandomizedSearchCV(LogisticRegression(random_state=42), param_logreg, n_iter=10,
                               scoring='accuracy', cv=5, random_state=42, n_jobs=-1)
rs_logreg.fit(X_train, y_train_encoded)

rs_tree = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), param_tree, n_iter=10,
                             scoring='accuracy', cv=5, random_state=42, n_jobs=-1)
rs_tree.fit(X_train, y_train_encoded)

rs_rf = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_rf, n_iter=10,
                           scoring='accuracy', cv=5, random_state=42, n_jobs=-1)
rs_rf.fit(X_train, y_train_encoded)

rs_xgb = RandomizedSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
                            param_xgb, n_iter=10, scoring='accuracy', cv=5, random_state=42, n_jobs=-1)
rs_xgb.fit(X_train, y_train_encoded)

rs_mlp = RandomizedSearchCV(MLPClassifier(random_state=42), param_mlp, n_iter=10,
                            scoring='accuracy', cv=5, random_state=42, n_jobs=-1)
rs_mlp.fit(X_train, y_train_encoded)

for nome, busca in {
    "Regressão Logística (Ot)": rs_logreg,
    "Árvore de Decisão (Ot)": rs_tree,
    "Random Forest (Ot)": rs_rf,
    "XGBoost (Ot)": rs_xgb,
    "Rede Neural (MLP Ot)": rs_mlp
}.items():
    modelo = busca.best_estimator_
    melhores_modelos[nome] = modelo

    pred_train = modelo.predict(X_train)
    pred_val = modelo.predict(X_val)

    models_optimazed.append(modelo)

    resultados_otimizados[nome] = {
        "Accuracy treino": accuracy_score(y_train_encoded, pred_train),
        "Accuracy validação": accuracy_score(y_val_encoded, pred_val),
        "Relatório de classificação": classification_report(y_val_encoded, pred_val, output_dict=True, zero_division=0)
    }

df_otimizados = pd.DataFrame({
    nome: {
        "Accuracy treino": valores["Accuracy treino"],
        "Accuracy validação": valores["Accuracy validação"]
    }
    for nome, valores in resultados_otimizados.items()
}).T.sort_values("Accuracy validação", ascending=False)

df_otimizados

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:51:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


,Accuracy treino,Accuracy validação
Regressão Logística (Ot),1.0,1.0
Árvore de Decisão (Ot),1.0,1.0
Random Forest (Ot),1.0,1.0
XGBoost (Ot),1.0,1.0
Rede Neural (MLP Ot),1.0,1.0


In [334]:
print("\nModelos manualmente definidos:")
for model in models:
  print(f"\n-> Avaliação {model.__class__.__name__}:")
  print("\nAccuracy treino:", accuracy_score(y_train, model.predict(X_train)))
  print("Accuracy validação:", accuracy_score(y_val, model.predict(X_val)))
  print("\nRelatório completo:\n", classification_report(y_val, model.predict(X_val)))

print("\nModelos otimizados:")
for model in models_optimazed:
  print(f"\n-> Avaliação {model.__class__.__name__}:")
  print("\nAccuracy treino:", accuracy_score(y_train_encoded, model.predict(X_train)))
  print("Accuracy validação:", accuracy_score(y_val_encoded, model.predict(X_val)))
  print("\nRelatório completo:\n", classification_report(y_val_encoded, model.predict(X_val)))


Modelos manualmente definidos:

-> Avaliação LogisticRegression:

Accuracy treino: 1.0
Accuracy validação: 1.0

Relatório completo:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       429
           1       1.00      1.00      1.00       495

    accuracy                           1.00       924
   macro avg       1.00      1.00      1.00       924
weighted avg       1.00      1.00      1.00       924


-> Avaliação DecisionTreeClassifier:

Accuracy treino: 1.0
Accuracy validação: 1.0

Relatório completo:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       429
           1       1.00      1.00      1.00       495

    accuracy                           1.00       924
   macro avg       1.00      1.00      1.00       924
weighted avg       1.00      1.00      1.00       924


-> Avaliação RandomForestClassifier:

Accuracy treino: 1.0
Accuracy validação: 1.0

Relatório completo:
 